<a href="https://colab.research.google.com/github/gustavo-ifusp/prefeitura_smit_sp360/blob/main/CNES_base_estabelecimentos_de_saude_FATO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

#**CNES - Cadastro Nacional de Estabelecimentos de Saúde - FATO  e dimensões do fato que necessitarão de atualização pela DAG**
**Link dos Dados de origem:**<br>
http://cnes.datasus.gov.br/pages/downloads/arquivosBaseDados.jsp<br><br>

**Dimensões estáticas:** <br>https://colab.research.google.com/drive/1iix3IPLXAoTrG3AlhW3EGcf90ZQC4saL?usp=sharing
<br><br>
Responsável: Gustavo de Andrade <br> gustavo.andrade@prefeitura.sp.gov.br



**tbEstabelecimento202108**

In [ ]:
dados = pd.read_csv("tbEstabelecimento202108.csv", sep = ';', header = 0, decimal = ',', parse_dates=True, encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,13,14,15,36,38,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dados

,CO_UNIDADE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO
0,CE00000000000000000000000018970,2527111,7.539273e+12,3,3,PREFEITURA MUNICIPAL DE VARZEA ALEGRE,CENTRO DE ASSISTENCIA A SAUDE DA FAMILIA,RUA FIGUEIREDO CORREIA,261,NaN,CENTRO,62590000,NaN,NaN,NaN,NaN,(88)667 1357,NaN,NaN,NaN,NaN,4,3.0,NaN,NaN,NaN,NaN,NaN,2,3.0,23,230655,01/03/2002,georgia,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,S,12/03/2002,NaN,NaN,NaN
1,MG0000312710000000000000180939,2195569,1.844913e+13,3,3,PREFEITURA MUNICIPAL DE FRUTAL,PSF BAIRRO XV DE NOVEMBRO PRINCESA IZABEL,RUA JOSE DO PATROCINIO,1043,NaN,XV DE NOVEMBRO,38200000,NaN,NaN,NaN,NaN,(34)34233371,(34)34219114,smph-scaa@knrt.com.br,NaN,NaN,4,2.0,NaN,NaN,NaN,NaN,NaN,2,3.0,31,312710,11/10/2002,mary lane,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,M,07/10/2002,NaN,NaN,NaN
2,RO00000000000000099990010000196,2497573,4.632212e+12,3,3,PREFEITURA MUNICIPAL DE PRESIDENTE MEDICI,POSTO DE SAUDE CRISTO REI PRESIDENTE MEDICI,LINHA 132,S/N,NaN,ZONA RURAL,78968000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1.0,NaN,NaN,NaN,NaN,NaN,1,3.0,11,110025,29/12/2001,caio,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,M,30/01/2002,NaN,NaN,NaN
3,AP00000000000000099990010000006,2021390,5.995766e+12,3,3,PREFEITURA MUNICIPAL DE MACAPA,PM MACAPA POSTO DE SAUDE FOZ DO MACACOARI,MACACOARI BAILIQUE,01,NaN,MACACOARI,68912350,001,NaN,NaN,NaN,(96)2131109,(96)2131009,NaN,NaN,NaN,4,1.0,NaN,NaN,NaN,NaN,NaN,1,3.0,16,160030,09/10/2002,eladia,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,M,09/11/2001,NaN,NaN,NaN
4,CE0000231290000000000000047929,2424231,7.598634e+12,3,3,PREFEITURA MUNICIPAL DE SOBRAL,PSF TERCEIRO BATALHAO,RUA FREDERICO GOMES,1030,NaN,CENTRO,62011000,0011,NaN,NaN,NaN,(88)6115895,NaN,NaN,NaN,NaN,4,3.0,346,12-jul-2001 00:00:00,2.0,NaN,NaN,2,3.0,23,231290,05/09/2002,sesa,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,M,28/04/2004,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442421,4318006560199,6560199,NaN,1,1,RAFAELA LIMA POERSCKE,CONS MED RAFAELA LIMA POERSCKE,AV PRESIDENTE VARGAS,1655,NaN,CENTRO,97670000,12,NaN,NaN,NaN,5534314343,NaN,NaN,8.916859e+10,NaN,4,3.0,940,01-jun-2010 00:00:00,2.0,NaN,NaN,22,3.0,43,431800,24/04/2019,SAUDE,8.916859e+10,31538,NaN,NaN,NaN,-28.661,-56.004,24/04/2019,SAUDE,4000.0,N,NaN,S,NaN,NaN,M,04/08/2010,16.0,1.0,NaN
442422,5003256567568,6567568,1.538960e+13,3,3,PREFEITRUA MUNICIPAL DE COSTA RICA,UNIDADE SAUDE DA FAMILIA RURAL,RUA J,171,NaN,VALE DO AMANHECER,79550000,0008,NaN,NaN,NaN,(67)3247-2168,NaN,SAUDECR@TERRA.COM.BR,NaN,NaN,4,1.0,502,02-ago-2010 00:00:00,2.0,NaN,NaN,2,3.0,50,500325,06/08/2019,CINTHIA,1.394560e+09,356079,NaN,4.0,NaN,-18.523826592522294,-53.14953804016113,06/08/2019,CINTHIA,1244.0,N,NaN,S,NaN,NaN,M,04/08/2010,1.0,12.0,NaN
442423,5107906579779,6579779,1.502400e+13,3,3,PREFEITURA MUNICIPAL DE SINOP,CENTRO DE REFERENCIA EM HANSENIASE E TUBERCULO...,RUA DOS EUCALIPTOS,208,PREDIO,SETOR COMERCIAL,78550180,014,NaN,NaN,

#**Tabela Fato Estabelecimento: cnes_fato_estabelecimento**

Com a tabela fato da base CNES, primeiro filtramos o **CO_MUNICIPIO_GESTOR** de São Paulo cujo código é **'355030'**

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442426 entries, 0 to 442425
Data columns (total 54 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   CO_UNIDADE                                   442426 non-null  object 
 1   CO_CNES                                      442426 non-null  int64  
 2   NU_CNPJ_MANTENEDORA                          104398 non-null  float64
 3   TP_PFPJ                                      442426 non-null  int64  
 4   NIVEL_DEP                                    442426 non-null  int64  
 5   NO_RAZAO_SOCIAL                              442424 non-null  object 
 6   NO_FANTASIA                                  442392 non-null  object 
 7   NO_LOGRADOURO                                442426 non-null  object 
 8   NU_ENDERECO                                  442132 non-null  object 
 9   NO_COMPLEMENTO                               200065 non-nul

In [ ]:
dados = dados.loc[lambda dados: dados["CO_MUNICIPIO_GESTOR"] == 355030]

In [ ]:
dados.drop(columns =['NO_COMPLEMENTO','CO_CLIENTELA','TP_ORGAO_EXPEDIDOR','CO_TURNO_ATENDIMENTO','ST_CONTRATO_FORMALIZADO','NU_CNPJ_MANTENEDORA','TP_PFPJ','NIVEL_DEP','CO_REGIAO_SAUDE','CO_MICRO_REGIAO','CO_DISTRITO_SANITARIO','CO_DISTRITO_ADMINISTRATIVO','NU_TELEFONE','NU_FAX','NO_EMAIL','NU_CPF','NU_CNPJ','NU_ALVARA','DT_VAL_LIC_SANI','TP_LIC_SANI','CO_ESTADO_GESTOR','CO_MUNICIPIO_GESTOR','CO_USUARIO','CO_CPFDIRETORCLN','REG_DIRETORCLN','ST_ADESAO_FILANTROP','NO_URL',"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",'NO_USUARIO_GEO','ST_GERACREDITO_GERENTE_SGIF','NO_FANTASIA_ABREV'], inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
dados = dados.reset_index(drop=True)

**Verificando repetidos**

In [ ]:
repetidos_verifica = pd.DataFrame()
repetidos_verifica['id_razao_social'] = dados.groupby(['NO_RAZAO_SOCIAL']).size().sort_values(ascending=False)

In [ ]:
repetidos_verifica.head(15)

,id_razao_social
NO_RAZAO_SOCIAL,
PREFEITURA DO MUNICIPIO DE SAO PAULO,1193
RAIA DROGASIL S A,130
DIAGNOSTICOS DA AMERICA S A,123
SECRETARIA DE ESTADO DA SAUDE DE SAO PAULO,84
FLEURY S A,64
DROGARIA SAO PAULO S A,50
DIAGNOSTICOS DA AMERICA SA,50
PREVENT SENIOR PRIVATE OPERADORA DE SAUDE LTDA,49
EMPREENDIMENTOS PAGUE MENOS S A,36


#**1) cnes_dim_estabelecimentos_razao_social**
<br>
Atualizar na DAG

In [ ]:
cnes_dim_estabelecimentos_razao_social = pd.DataFrame()
cnes_dim_estabelecimentos_razao_social['razao_social'] = dados['NO_RAZAO_SOCIAL'].drop_duplicates()
cnes_dim_estabelecimentos_razao_social = cnes_dim_estabelecimentos_razao_social.sort_values(by=['razao_social'], ascending=True)
cnes_dim_estabelecimentos_razao_social.index = np.arange(1, len(cnes_dim_estabelecimentos_razao_social) + 1)
cnes_dim_estabelecimentos_razao_social['id_razao_social'] = cnes_dim_estabelecimentos_razao_social.index

In [ ]:
cnes_dim_estabelecimentos_razao_social

,razao_social,id_razao_social
1,021 DENTAL AVENIDA PAULISTA ODONTOLOGIA LTDA,1
2,021 DENTAL SAO BENTO ODONTOLOGIA LTDA,2
3,021 DENTAL VILA OLIMPIA ODONTOLOGIA LTDA,3
4,11CARE SERVICOS DE SAUDE LTDA,4
5,2 PILARES TERAPEUTICOS PSICOLOGIA LTDA ME,5
...,...,...
22816,ZULEICA SOARES,22816
22817,ZULEIKA RIBEIRO PINHEIRO MACHADO,22817
22818,ZULMIRA MARTINEZ MONTIEL,22818
22819,ZVEIBIL E FERNANDES ODONTOLOGIA LTDA,22819


In [ ]:
repetidos_verifica = pd.DataFrame()
repetidos_verifica['soma_verifica'] = cnes_dim_estabelecimentos_razao_social.groupby(['razao_social']).size().sort_values(ascending=False)

In [ ]:
repetidos_verifica

,soma_verifica
razao_social,
ZW ODONTOLOGIA LTDA,1
DAVITA SERVICOS DE NEFROLOGIA JARDIM DAS IMBUIAS LTDA,1
DAVI KORN,1
DAVI RUBIO,1
DAVID CHVINDELMAN,1
...,...
MARIA JOSE AKAMINE,1
MARIA JOSE BERGAMIN,1
MARIA JOSE CAIUBY GUIMARAES PSICOLOGIA ME,1


In [ ]:
n = a = 0
ver = False
while n < len(repetidos_verifica):
    a = int(repetidos_verifica['soma_verifica'][n])
    if a != 1: 
        ver = True
    n += 1
if ver is True:
    print("ERRO! existe entradas repetidas.")
else:
    print("SUCESSO! Não existe entradas repetidas")

SUCESSO! Não existe entradas repetidas


In [ ]:
cnes_dim_estabelecimentos_razao_social.to_csv(r'cnes_dim_estabelecimentos_razao_social.csv', index = False)

#**Dicionário Razao Social**

In [ ]:
cnes_dim_estabelecimentos_razao_social.values

array([['021 DENTAL AVENIDA PAULISTA ODONTOLOGIA LTDA', 1],
       ['021 DENTAL SAO BENTO ODONTOLOGIA LTDA', 2],
       ['021 DENTAL VILA OLIMPIA ODONTOLOGIA LTDA', 3],
       ...,
       ['ZULMIRA MARTINEZ MONTIEL', 22818],
       ['ZVEIBIL E FERNANDES ODONTOLOGIA LTDA', 22819],
       ['ZW ODONTOLOGIA LTDA', 22820]], dtype=object)

In [ ]:
sorted(cnes_dim_estabelecimentos_razao_social.values.tolist())

[['021 DENTAL AVENIDA PAULISTA ODONTOLOGIA LTDA', 1],
 ['021 DENTAL SAO BENTO ODONTOLOGIA LTDA', 2],
 ['021 DENTAL VILA OLIMPIA ODONTOLOGIA LTDA', 3],
 ['11CARE SERVICOS DE SAUDE LTDA', 4],
 ['2 PILARES TERAPEUTICOS PSICOLOGIA LTDA ME', 5],
 ['2020 SERVICOS MEDICOS SS', 6],
 ['2CSJ MEDICOS ASSOCIADOS LTDA', 7],
 ['2L SERVICOS MEDICOS LTDA', 8],
 ['2N CLINICA ODONTOLOGICA ESPECIALIZADA LTDA', 9],
 ['3 COLINAS ODONTOLOGIA LTDA', 10],
 ['3 E SERVICOS MEDICOS E PARTICIPACOES LTDA', 11],
 ['3 K SERVICOS MEDICOS LTDA EPP', 12],
 ['3D ODONTOLOGIA INTEGRADA LTDA', 13],
 ['4 MAIS ODONTOLOGIA INTEGRADA LTDA', 14],
 ['4ANTUNES CONSULTORIO ODONTOLOGICO LTDA', 15],
 ['4BIO MEDICAMENTOS SA', 16],
 ['4F CLINICA ODONTOLOGICA', 17],
 ['5IVE ODONTOLOGIA INTEGRADA LTDA ME', 18],
 ['7NA FONOAUDIOLOGIA E TECNOLOGIA LTDA', 19],
 ['9 DE JULHO ENDOSCOPIA', 20],
 ['A A MARTINO ARTI DENTALI CLINICA ODONTOLOGICA LTDA', 21],
 ['A B C D ASSOCIACAO BRASILEIRA DE COLITE ULCERATIVA E DOENCA', 22],
 ['A C A DO NASCIME

In [ ]:
dict(sorted(cnes_dim_estabelecimentos_razao_social.values.tolist())) 

{'021 DENTAL AVENIDA PAULISTA ODONTOLOGIA LTDA': 1,
 '021 DENTAL SAO BENTO ODONTOLOGIA LTDA': 2,
 '021 DENTAL VILA OLIMPIA ODONTOLOGIA LTDA': 3,
 '11CARE SERVICOS DE SAUDE LTDA': 4,
 '2 PILARES TERAPEUTICOS PSICOLOGIA LTDA ME': 5,
 '2020 SERVICOS MEDICOS SS': 6,
 '2CSJ MEDICOS ASSOCIADOS LTDA': 7,
 '2L SERVICOS MEDICOS LTDA': 8,
 '2N CLINICA ODONTOLOGICA ESPECIALIZADA LTDA': 9,
 '3 COLINAS ODONTOLOGIA LTDA': 10,
 '3 E SERVICOS MEDICOS E PARTICIPACOES LTDA': 11,
 '3 K SERVICOS MEDICOS LTDA EPP': 12,
 '3D ODONTOLOGIA INTEGRADA LTDA': 13,
 '4 MAIS ODONTOLOGIA INTEGRADA LTDA': 14,
 '4ANTUNES CONSULTORIO ODONTOLOGICO LTDA': 15,
 '4BIO MEDICAMENTOS SA': 16,
 '4F CLINICA ODONTOLOGICA': 17,
 '5IVE ODONTOLOGIA INTEGRADA LTDA ME': 18,
 '7NA FONOAUDIOLOGIA E TECNOLOGIA LTDA': 19,
 '9 DE JULHO ENDOSCOPIA': 20,
 'A A MARTINO ARTI DENTALI CLINICA ODONTOLOGICA LTDA': 21,
 'A B C D ASSOCIACAO BRASILEIRA DE COLITE ULCERATIVA E DOENCA': 22,
 'A C A DO NASCIMENTO ODONTOLOGIA': 23,
 'A C ASSISTENCIA MEDIC

In [ ]:
def ConverteRazaoSocialNumID(id_razao_social):
    dicionario_razao_social = dict(sorted(cnes_dim_estabelecimentos_razao_social.values.tolist())) 
    return dicionario_razao_social[id_razao_social]

#**Aplicando dicionário Nome Razão Social**

In [ ]:
n = 0
lista = []
while n < len(dados): 
  id_razao_social = dados['NO_RAZAO_SOCIAL'][n]
  razao_social = ConverteRazaoSocialNumID(id_razao_social)
  lista.insert(n, razao_social)
  n += 1
print(lista)

[5815, 10906, 12745, 15346, 20554, 13789, 17630, 8962, 856, 21225, 15998, 923, 7984, 18632, 823, 2159, 22517, 16105, 5097, 19043, 6634, 12207, 5979, 2679, 11487, 3366, 7732, 7246, 3099, 1873, 18659, 18188, 36, 2375, 12894, 4970, 15585, 19768, 4039, 16970, 832, 4840, 17775, 18632, 17074, 18632, 20969, 14028, 16947, 3547, 2295, 16994, 18632, 20868, 5189, 12702, 18548, 10381, 22478, 4586, 14652, 8336, 16293, 18632, 6572, 8336, 20031, 15454, 13907, 4, 7663, 20760, 18215, 20103, 17951, 18215, 11866, 20102, 17589, 20654, 1972, 20848, 14348, 21631, 707, 14381, 4911, 19753, 22114, 16960, 2197, 8968, 13264, 9211, 16881, 12853, 5360, 11454, 21640, 15866, 2165, 3072, 2045, 13384, 22352, 8393, 7071, 13507, 605, 8722, 13757, 13150, 5360, 13486, 11069, 9108, 2244, 9374, 10074, 9725, 14340, 10422, 13142, 20628, 6689, 1194, 18351, 19382, 11542, 18858, 16705, 2313, 11551, 7860, 13299, 19701, 22359, 2063, 4073, 14461, 20590, 9970, 12298, 5243, 4834, 16488, 18215, 20942, 10005, 5589, 15929, 14472, 18863,

In [ ]:
dados['NO_RAZAO_SOCIAL'] = lista

In [ ]:
dados = dados.rename(columns = {'NO_RAZAO_SOCIAL': 'id_razao_social'}, inplace = False)
dados

,CO_UNIDADE,CO_CNES,id_razao_social,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,CO_ATIVIDADE,DT_EXPEDICAO,TP_UNIDADE,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_MOTIVO_DESAB,NU_LATITUDE,NU_LONGITUDE,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL
0,SP00003509205000001329730000101,2825287,5815,CLINICA MEDICA SULLA PELLE S C LTDA,RUA PAES DE ANDRADE,929,ACLIMACAO,1530000,4,NaN,4,09/09/2003,8.0,NaN,NaN,2000.0,NaN,NaN,NaN,M,05/11/2003,NaN,NaN
1,3550309902198,9902198,10906,CLINICA DE CIRURGIA PLASTICA OMNIA,RUA JOAO LOURENCO,766,VILA NOVA CONCEICAO,4508031,4,28-ago-2019 00:00:00,36,04/09/2019,NaN,-23.590627,-46.6682989,2240.0,N,S,NaN,M,09/09/2019,16.0,1.0
2,3550309902309,9902309,12745,JSMC SERVICOS MEDICOS LTDA,RUA CRISTIANO VIANA,401,PINHEIROS,5406050,4,21-nov-2019 00:00:00,22,24/11/2019,NaN,-23.5592723,-46.677217,2240.0,N,S,NaN,M,09/09/2019,16.0,1.0
3,3550300868663,868663,15346,CONS ODONTOLOGICO MARIANA GABRIEL DE SOUZA,RUA VATICANO,126,JD JOAO XXIII,5569120,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6179578,-46.5353544,4000.0,N,S,NaN,M,22/08/2021,16.0,1.0
4,3550300868655,868655,20554,CONS ODONTO SIMONE BAEZO RASSY,RUA JOSE RAMON URTIZA,837,VILA ANDRADE,5717270,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6349407,-46.7271941,4000.0,N,S,NaN,M,22/08/2021,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26137,3550306749755,6749755,19076,RIBEIRO E DELFINO SERVICOS MEDICOS,AVENIDA FAGUNDES FILHO,191,SAO JUDAS TADEU,4304010,4,01-mai-2010 00:00:00,22,05/02/2019,8.0,NaN,NaN,2240.0,NaN,S,NaN,M,01/05/2011,16.0,1.0
26138,3550306749771,6749771,21088,CONS ODONTO SUSAN MARTINEZ CASTANHO,RUA BRAULIO GOMES,25,CENTRO,1047020,4,22-mar-2011 00:00:00,22,28/04/2011,8.0,NaN,NaN,4000.0,NaN,NaN,NaN,M,01/05/2011,16.0,1.0
26139,3550306751040,6751040,22652,CONS ODONTO WILLIAM E WILLIAM ODONTOLOGIA,RUA BARAO DO TRIUNFO,1770,CAMPO BELO,4602006,4,06-jan-2009 00:00:00,22,13/02/2019,8.0,NaN,NaN,2062.0,NaN,S,NaN,M,01/05/2011,16.0,1.0
26140,3550306751008,6751008,22480,VIZIO CLINICA OFTALMOLOGICA,RUA DOUTOR ALCEU DE CAMPOS RODRIGUES,229,VILA NOVA CONCEICAO,4544000,4,NaN,36,20/08/2021,NaN,-23.5912333,-46.6765755,2062.0,N,S,NaN,M,01/05/2011,16.0,1.0


#**2) cnes_dim_estabelecimentos_nome_fantasia**
<br>
Atualizar na DAG

In [ ]:
cnes_dim_estabelecimentos_nome_fantasia = pd.DataFrame()
cnes_dim_estabelecimentos_nome_fantasia['nome_fantasia'] = dados['NO_FANTASIA'].drop_duplicates()
cnes_dim_estabelecimentos_nome_fantasia = cnes_dim_estabelecimentos_nome_fantasia.sort_values(by=['nome_fantasia'], ascending=True)
cnes_dim_estabelecimentos_nome_fantasia.index = np.arange(1, len(cnes_dim_estabelecimentos_nome_fantasia) + 1)
cnes_dim_estabelecimentos_nome_fantasia['id_razao_social'] = cnes_dim_estabelecimentos_nome_fantasia.index

In [ ]:
cnes_dim_estabelecimentos_nome_fantasia

,nome_fantasia,id_razao_social
1,021 DENTAL AVENIDA PAULISTA ODONTOLOGIA,1
2,021 DENTAL SAO BENTO ODONTOLOGIA,2
3,021 DENTAL VILA OLIMPIA ODONTOLOGIA,3
4,11CARE,4
5,2 PILARES TERAPEUTICOS,5
...,...,...
25528,ZORTEA E PEREIRA CLINICAS ODONTOLOGICAS,25528
25529,ZORZI E GASPAR CONSULTORIO ODONTOLOGICO,25529
25530,ZRS SERVICOS MEDICOS,25530
25531,ZVEIBIL E FERNANDES ODONTOLOGIA,25531


In [ ]:
cnes_dim_estabelecimentos_nome_fantasia.to_csv(r'cnes_dim_estabelecimentos_nome_fantasia.csv', index = False)

In [ ]:
repetidos_verifica = pd.DataFrame()
repetidos_verifica['soma_verifica'] = cnes_dim_estabelecimentos_nome_fantasia.groupby(['nome_fantasia']).size().sort_values(ascending=False)
n = a = 0
ver = False
while n < len(repetidos_verifica):
    a = int(repetidos_verifica['soma_verifica'][n])
    if a != 1: 
        ver = True
    n += 1
if ver is True:
    print("ERRO! existe entradas repetidas.")
else:
    print("SUCESSO! Não existe entradas repetidas")

SUCESSO! Não existe entradas repetidas


**Criar uma função na dag de verificar_entradas_repetidas o máximo genérica para chama-la várias vezes - Tarefinha**

#**Dicionário Nome Fantasia**

In [ ]:
def ConverteNomeFantasiaNumID(id_nome_fantasia):
    dicionario_nome_fantasia = dict(sorted(cnes_dim_estabelecimentos_nome_fantasia.values.tolist())) 
    return dicionario_nome_fantasia[id_nome_fantasia]

#**Aplicando dicionário Nome Fantasia**

In [ ]:
n = 0
lista = []
while n < len(dados): 
  id_nome_fantasia = dados['NO_FANTASIA'][n]
  nome_fantasia = ConverteNomeFantasiaNumID(id_nome_fantasia)
  lista.insert(n, nome_fantasia)
  n += 1
print(lista)

[5117, 3324, 20258, 15676, 15082, 20810, 22509, 17995, 6501, 5906, 21389, 17846, 3699, 23094, 664, 1173, 25419, 14040, 3794, 25320, 3229, 20154, 11166, 11183, 5286, 2152, 6063, 11570, 11339, 977, 6667, 22737, 34, 1502, 20274, 3693, 16213, 23421, 2859, 21901, 3137, 3564, 16271, 23103, 22008, 23092, 25121, 16869, 21882, 2340, 7209, 21932, 23088, 23975, 4410, 19828, 23035, 18210, 25391, 3327, 13554, 17724, 21518, 17775, 6012, 17724, 23590, 13887, 20880, 4, 22567, 16380, 25051, 23649, 9542, 24601, 19962, 187, 14278, 23874, 1031, 15143, 20939, 24380, 370, 20964, 3636, 23414, 25198, 21891, 1379, 17999, 20527, 18166, 21833, 6350, 20391, 2275, 24390, 21270, 1179, 23227, 1089, 21270, 25336, 17873, 16973, 20735, 21656, 1395, 20797, 20342, 20391, 20727, 12597, 22239, 1396, 21277, 4466, 5294, 20934, 18823, 20339, 23855, 6150, 6962, 22848, 23139, 19775, 9659, 21722, 1444, 19646, 1023, 17501, 23384, 25343, 1103, 5952, 21007, 23835, 5366, 20179, 3959, 3555, 21623, 2322, 24072, 2846, 17622, 21333, 210

In [ ]:
dados['NO_FANTASIA'] = lista

In [ ]:
dados = dados.rename(columns = {'NO_FANTASIA': 'id_nome_fantasia'}, inplace = False)

In [ ]:
dados

,CO_UNIDADE,CO_CNES,id_razao_social,id_nome_fantasia,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,CO_CEP,CO_ATIVIDADE,DT_EXPEDICAO,TP_UNIDADE,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_MOTIVO_DESAB,NU_LATITUDE,NU_LONGITUDE,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL
0,SP00003509205000001329730000101,2825287,5815,5117,RUA PAES DE ANDRADE,929,ACLIMACAO,1530000,4,NaN,4,09/09/2003,8.0,NaN,NaN,2000.0,NaN,NaN,NaN,M,05/11/2003,NaN,NaN
1,3550309902198,9902198,10906,3324,RUA JOAO LOURENCO,766,VILA NOVA CONCEICAO,4508031,4,28-ago-2019 00:00:00,36,04/09/2019,NaN,-23.590627,-46.6682989,2240.0,N,S,NaN,M,09/09/2019,16.0,1.0
2,3550309902309,9902309,12745,20258,RUA CRISTIANO VIANA,401,PINHEIROS,5406050,4,21-nov-2019 00:00:00,22,24/11/2019,NaN,-23.5592723,-46.677217,2240.0,N,S,NaN,M,09/09/2019,16.0,1.0
3,3550300868663,868663,15346,15676,RUA VATICANO,126,JD JOAO XXIII,5569120,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6179578,-46.5353544,4000.0,N,S,NaN,M,22/08/2021,16.0,1.0
4,3550300868655,868655,20554,15082,RUA JOSE RAMON URTIZA,837,VILA ANDRADE,5717270,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6349407,-46.7271941,4000.0,N,S,NaN,M,22/08/2021,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26137,3550306749755,6749755,19076,23265,AVENIDA FAGUNDES FILHO,191,SAO JUDAS TADEU,4304010,4,01-mai-2010 00:00:00,22,05/02/2019,8.0,NaN,NaN,2240.0,NaN,S,NaN,M,01/05/2011,16.0,1.0
26138,3550306749771,6749771,21088,15185,RUA BRAULIO GOMES,25,CENTRO,1047020,4,22-mar-2011 00:00:00,22,28/04/2011,8.0,NaN,NaN,4000.0,NaN,NaN,NaN,M,01/05/2011,16.0,1.0
26139,3550306751040,6751040,22652,15599,RUA BARAO DO TRIUNFO,1770,CAMPO BELO,4602006,4,06-jan-2009 00:00:00,22,13/02/2019,8.0,NaN,NaN,2062.0,NaN,S,NaN,M,01/05/2011,16.0,1.0
26140,3550306751008,6751008,22480,25393,RUA DOUTOR ALCEU DE CAMPOS RODRIGUES,229,VILA NOVA CONCEICAO,4544000,4,NaN,36,20/08/2021,NaN,-23.5912333,-46.6765755,2062.0,N,S,NaN,M,01/05/2011,16.0,1.0


#**3) cnes_dim_estabelecimentos_nome_logradouro**
<br>
Atualizar na DAG

In [ ]:
cnes_dim_estabelecimentos_nome_logradouro = pd.DataFrame()
cnes_dim_estabelecimentos_nome_logradouro['nome_logradouro'] = dados['NO_LOGRADOURO'].drop_duplicates()
cnes_dim_estabelecimentos_nome_logradouro = cnes_dim_estabelecimentos_nome_logradouro.sort_values(by=['nome_logradouro'], ascending=True)
cnes_dim_estabelecimentos_nome_logradouro.index = np.arange(1, len(cnes_dim_estabelecimentos_nome_logradouro) + 1)
cnes_dim_estabelecimentos_nome_logradouro['id_nome_logradouro'] = cnes_dim_estabelecimentos_nome_logradouro.index
cnes_dim_estabelecimentos_nome_logradouro

,nome_logradouro,id_nome_logradouro
1,2 DE JULHO,1
2,23 6168,2
3,46 6213526,3
4,A JOAQUIM EUGENEO DE LIMA,4
5,A V MATEO BEI,5
...,...,...
7634,VITAL BRASIL,7634
7635,VOLUNTARIOS DA PATRIA,7635
7636,WALDEMAR TIETZ,7636
7637,WASHINGTON LUIS,7637


In [ ]:
repetidos_verifica = pd.DataFrame()
repetidos_verifica['soma_verifica'] = cnes_dim_estabelecimentos_nome_logradouro.groupby(['nome_logradouro']).size().sort_values(ascending=False)

In [ ]:
n = a = 0
ver = False
while n < len(repetidos_verifica):
    a = int(repetidos_verifica['soma_verifica'][n])
    if a != 1: 
        ver = True
    n += 1
if ver is True:
    print("ERRO! existe entradas repetidas.")
else:
    print("SUCESSO! Não existe entradas repetidas")

SUCESSO! Não existe entradas repetidas


In [ ]:
cnes_dim_estabelecimentos_nome_logradouro.to_csv(r'cnes_dim_estabelecimentos_nome_logradouro.csv', index = False)

#**Dicionário Nome Logradouro**

In [ ]:
def ConverteNomeLogradouroNumID(id_nome_logradouro):
    dicionario_nome_logradouro = dict(sorted(cnes_dim_estabelecimentos_nome_logradouro.values.tolist())) 
    return dicionario_nome_logradouro[id_nome_logradouro]

#**Aplicando dicionário Nome Logradouro**

In [ ]:
n = 0
lista = []
while n < len(dados): 
  id_nome_logradouro = dados['NO_LOGRADOURO'][n]
  nome_logradouro = ConverteNomeLogradouroNumID(id_nome_logradouro)
  lista.insert(n, nome_logradouro)
  n += 1
print(lista)

[6547, 5768, 4498, 7377, 5891, 6891, 1335, 720, 6127, 2761, 1238, 1152, 3032, 6891, 6868, 6678, 6968, 97, 6114, 3002, 1055, 911, 5653, 6415, 3936, 4672, 4337, 680, 395, 6423, 4065, 5772, 4781, 1300, 1244, 56, 4238, 5282, 7388, 5015, 7133, 2729, 3644, 6181, 6672, 1144, 7388, 6460, 3987, 1416, 6506, 7388, 1387, 1119, 3797, 3142, 2287, 14, 6674, 5645, 4798, 706, 6647, 6460, 659, 2769, 4878, 5345, 605, 7511, 5952, 2387, 3339, 2463, 108, 1095, 509, 3545, 5080, 370, 3378, 412, 5573, 3836, 6807, 30, 4022, 2688, 3658, 4842, 6564, 944, 920, 2702, 572, 4930, 4481, 3653, 5536, 5426, 5644, 4305, 5573, 2770, 2187, 6626, 4420, 5201, 398, 4865, 5186, 2582, 7013, 4479, 4058, 5848, 465, 3316, 6564, 1345, 3142, 6068, 401, 5651, 299, 4270, 3344, 927, 5651, 5614, 658, 4297, 2117, 4532, 755, 3918, 7327, 468, 7388, 3108, 994, 3215, 2344, 7565, 5595, 1016, 4218, 3364, 2553, 1012, 6192, 465, 7212, 1276, 2804, 5045, 6534, 3938, 1320, 4043, 2453, 1086, 3865, 4880, 1379, 311, 5651, 6372, 1097, 829, 3675, 4019, 9

In [ ]:
dados['NO_LOGRADOURO'] = lista

In [ ]:
dados = dados.rename(columns = {'NO_FANTASIA': 'id_nome_fantasia'}, inplace = False)

#**4) cnes_dim_estabelecimentos_nome_bairro**
<br>
Atualizar na DAG

In [ ]:
cnes_dim_estabelecimentos_nome_bairro = pd.DataFrame()
cnes_dim_estabelecimentos_nome_bairro['nome_bairro'] = dados['NO_BAIRRO'].drop_duplicates()
cnes_dim_estabelecimentos_nome_bairro = cnes_dim_estabelecimentos_nome_bairro.sort_values(by=['nome_bairro'], ascending=True)
cnes_dim_estabelecimentos_nome_bairro['id_nome_bairro'] = cnes_dim_estabelecimentos_nome_bairro.index
cnes_dim_estabelecimentos_nome_bairro

,nome_bairro,id_nome_bairro
23581,01308050,23581
21280,03121020,21280
6978,310,6978
3727,A DE PINHEIROS,3727
21526,A E CARVALHO,21526
...,...,...
1288,VOLA CLEMENTINO,1288
5010,VVILA DOM PEDRO I,5010
10600,XV DE NOVEMBRO,10600
19195,ZONA LESTE,19195


In [ ]:
repetidos_verifica = pd.DataFrame()
repetidos_verifica['soma_verifica'] = cnes_dim_estabelecimentos_nome_bairro.groupby(['nome_bairro']).size().sort_values(ascending=False)
n = a = 0
ver = False
while n < len(repetidos_verifica):
    a = int(repetidos_verifica['soma_verifica'][n])
    if a != 1: 
        ver = True
    n += 1
if ver is True:
    print("ERRO! existe entradas repetidas.")
else:
    print("SUCESSO! Não existe entradas repetidas")

SUCESSO! Não existe entradas repetidas


In [ ]:
cnes_dim_estabelecimentos_nome_bairro.to_csv(r'cnes_dim_estabelecimentos_nome_bairro.csv', index = False)

#**Dicionário Nome Bairro**

In [ ]:
def ConverteNomeBairroNumID(id_nome_bairro):
    dicionario_nome_bairro = dict(sorted(cnes_dim_estabelecimentos_nome_bairro.values.tolist())) 
    return dicionario_nome_bairro[id_nome_bairro]

#**Aplicando dicionário No Bairro**

In [ ]:
n = 0
lista = []
while n < len(dados): 
  id_nome_bairro = dados['NO_BAIRRO'][n]
  nome_logradouro = ConverteNomeBairroNumID(id_nome_bairro)
  lista.insert(n, nome_logradouro)
  n += 1
print(lista)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 14, 15, 16, 10, 18, 19, 20, 21, 12, 23, 24, 14, 26, 0, 28, 26, 30, 2, 2, 33, 10, 35, 36, 37, 38, 39, 40, 38, 42, 43, 44, 45, 24, 47, 48, 8, 50, 38, 52, 23, 50, 55, 56, 57, 48, 59, 60, 61, 62, 2, 64, 65, 66, 67, 10, 69, 70, 12, 72, 40, 44, 75, 76, 77, 78, 0, 40, 81, 82, 83, 84, 10, 86, 87, 8, 82, 44, 1, 92, 87, 4, 78, 69, 48, 98, 48, 59, 101, 82, 62, 87, 57, 78, 107, 108, 109, 38, 2, 38, 113, 114, 6, 116, 23, 86, 119, 120, 62, 122, 123, 124, 12, 78, 127, 123, 129, 130, 8, 132, 133, 134, 135, 50, 137, 38, 24, 8, 141, 50, 24, 101, 145, 38, 147, 148, 149, 150, 48, 82, 153, 78, 18, 114, 42, 158, 159, 160, 84, 132, 1, 62, 2, 123, 19, 168, 123, 133, 171, 10, 173, 174, 175, 176, 82, 12, 179, 180, 26, 38, 82, 2, 185, 186, 133, 141, 82, 86, 4, 135, 193, 0, 1, 196, 10, 8, 199, 200, 201, 18, 123, 82, 40, 48, 98, 208, 123, 50, 62, 2, 50, 14, 38, 2, 18, 2, 11, 220, 38, 44, 223, 224, 0, 123, 227, 228, 86, 123, 173, 232, 233, 50, 78, 123, 208, 238, 239, 78

In [ ]:
dados['NO_BAIRRO'] = lista

In [ ]:
dados = dados.rename(columns = {'NO_BAIRRO': 'id_nome_bairro' }, inplace = False)

#**Tabela Fato Estabelecimento: cnes_fato_estabelecimento** - Modelagem

In [ ]:
dados = dados.rename(columns = {'CO_UNIDADE': 'co_unidade','CO_CNES':'co_cnes','NO_LOGRADOURO':'id_nome_logradouro','NU_ENDERECO':'numero_endereço','CO_CEP':'num_cep','TP_UNIDADE':'tipo_unidade'}, inplace = False)

In [ ]:
dados = dados.rename(columns = {'DT_EXPEDICAO':'data_expedicao',}, inplace = False)

In [ ]:
dados

,co_unidade,co_cnes,id_razao_social,id_nome_fantasia,id_nome_logradouro,numero_endereço,id_nome_bairro,num_cep,CO_ATIVIDADE,data_expedicao,tipo_unidade,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_MOTIVO_DESAB,NU_LATITUDE,NU_LONGITUDE,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL
0,SP00003509205000001329730000101,2825287,5815,5117,6547,929,0,1530000,4,NaN,4,09/09/2003,8.0,NaN,NaN,2000.0,NaN,NaN,NaN,M,05/11/2003,NaN,NaN
1,3550309902198,9902198,10906,3324,5768,766,1,4508031,4,28-ago-2019 00:00:00,36,04/09/2019,NaN,-23.590627,-46.6682989,2240.0,N,S,NaN,M,09/09/2019,16.0,1.0
2,3550309902309,9902309,12745,20258,4498,401,2,5406050,4,21-nov-2019 00:00:00,22,24/11/2019,NaN,-23.5592723,-46.677217,2240.0,N,S,NaN,M,09/09/2019,16.0,1.0
3,3550300868663,868663,15346,15676,7377,126,3,5569120,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6179578,-46.5353544,4000.0,N,S,NaN,M,22/08/2021,16.0,1.0
4,3550300868655,868655,20554,15082,5891,837,4,5717270,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6349407,-46.7271941,4000.0,N,S,NaN,M,22/08/2021,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26137,3550306749755,6749755,19076,23265,1207,191,3417,4304010,4,01-mai-2010 00:00:00,22,05/02/2019,8.0,NaN,NaN,2240.0,NaN,S,NaN,M,01/05/2011,16.0,1.0
26138,3550306749771,6749771,21088,15185,4081,25,14,1047020,4,22-mar-2011 00:00:00,22,28/04/2011,8.0,NaN,NaN,4000.0,NaN,NaN,NaN,M,01/05/2011,16.0,1.0
26139,3550306751040,6751040,22652,15599,3951,1770,124,4602006,4,06-jan-2009 00:00:00,22,13/02/2019,8.0,NaN,NaN,2062.0,NaN,S,NaN,M,01/05/2011,16.0,1.0
26140,3550306751008,6751008,22480,25393,4797,229,1,4544000,4,NaN,36,20/08/2021,NaN,-23.5912333,-46.6765755,2062.0,N,S,NaN,M,01/05/2011,16.0,1.0


In [ ]:
dados = cnes_fato_estabelecimento

In [ ]:
cnes_fato_estabelecimento.to_csv(r'cnes_fato_estabelecimento.csv', index = False)

#**CO_ATIVIDADE = CO_TIPO**
Arquivo original: **tbAtividade202108**

In [ ]:
a = n = 0
L = []
while n < len(dados):
    a = int(dados['CO_ATIVIDADE'][n])
    L.append(a)
    n += 1
print(L)

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [ ]:
a = n = 0
L = []
while n < len(dados):
    a = int(dados['id_atividade'][n])
    if a != 4:
        L.append(a)
    n += 1
print(L)

[3, 1, 2, 1, 1, 1, 1, 3, 2, 3, 3, 3, 1, 3, 2, 3, 3, 3, 2, 3, 1, 3, 2, 3, 1, 1, 3, 2, 1, 3, 2, 3, 1, 3, 5, 3, 3, 5, 1, 3, 3, 3, 2, 1, 3, 1, 1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 1, 2, 1, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 5, 2, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 5, 3, 2, 2, 2, 1, 3, 3, 1, 1, 1, 1, 2, 1, 2, 1, 3, 3, 3, 3, 2, 2, 3, 1, 1, 3, 1, 1, 3, 5, 1, 1, 3, 1, 1, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 5, 3, 3, 3, 3, 1, 3, 2, 5, 1, 2, 3, 3, 3, 2, 3, 2, 3, 2, 3, 5, 3, 3, 1, 3, 3, 1, 3, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 5, 3, 1, 1, 2, 2, 2, 1, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 5, 3, 3, 2, 5, 1, 2, 1, 3, 1, 3, 1, 3, 2, 2, 3, 1, 1, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 5, 2, 3, 5, 3, 3, 3, 3, 2, 3]


In [ ]:
cnes_fato_estabelecimento = pd.read_csv("cnes_fato_estabelecimento.csv")

In [ ]:
dim_atividade = pd.read_csv("tbAtividade202108.csv", sep = ';', header = 0, decimal = ',', parse_dates=True, encoding='utf-8')

In [ ]:
cnes_fato_estabelecimento  = cnes_fato_estabelecimento.rename(columns = {'CO_ATIVIDADE':'id_atividade'}, inplace = False)

In [ ]:
cnes_dim_atividade  = cnes_dim_atividade.rename(columns = {'CO_ATIVIDADE':'id_atividade', 'DS_ATIVIDADE':'atividade','DS_CONCEITO_ATIVIDADE':'conceito_atividade'	}, inplace = False)

In [ ]:
cnes_dim_atividade.drop(columns =['CO_GRUPO_ATIVIDADE',], inplace = True) 

In [ ]:
cnes_dim_atividade

,id_atividade,atividade,conceito_atividade
0,19,ANALISES LABORATORIAIS DE VIGILANCIA EM SAUDE,Ações e serviços de análises laboratoriais de ...
1,20,VIGILANCIA EM SAUDE DO TRABALHADOR,"Ações e serviços de vigilância em saúde, clíni..."
2,21,ESCLARECIMENTO DA CAUSA MORTIS DE OBITOS,Ações e serviços de realização de autópsia par...
3,22,ADMINISTRACAO,"Compreende os processos de formulação, impleme..."
4,23,REGULACAO ASSISTENCIAL,"Compreende a organização, o controle, o gerenc..."
5,24,LOGISTICA DE INSUMOS,"Compreende o armazenamento e distribuição, sem..."
6,25,"LOGISTICA DE ORGAOS, TECIDOS E CELULAS DO CORP...","Compreende as ações para o acondicionamento, g..."
7,26,HOSPITALIDADE,Serviços que visam alojar temporariamente e ap...
8,27,CONFECCAO DE ORTESES E PROTESES DENTARIAS,Serviço de fabricação e produção de órteses e ...
9,0,NAO SE APLICA,Sem Conceito


In [ ]:
cnes_dim_atividade.to_csv(r'cnes_dim_atividade.csv', index = False)

In [ ]:
cnes_fato_estabelecimento = cnes_fato_estabelecimento.rename(columns = {"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')":"data_ultima_atualizacao",'NU_LATITUDE':'latitude','NU_LONGITUDE':'longitude','CO_NATUREZA_JUR':'id_natureza_juridica', 'TP_ESTAB_SEMPRE_ABERTO':'sempre_aberto','ST_CONEXAO_INTERNET':'conexao_internet', 'CO_TIPO_UNIDADE':'tipo_unidade','TP_GESTAO':'gestao','CO_TIPO_ESTABELECIMENTO':'id_tipo_estabelecimento','CO_ATIVIDADE_PRINCIPAL':'id_atividade_principal'}, inplace = False)

In [ ]:
cnes_fato_estabelecimento.drop(columns =["TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",], inplace = True) 

In [ ]:
cnes_fato_estabelecimento = cnes_fato_estabelecimento.rename(columns = {'CO_MOTIVO_DESAB':'motivo_desativo'}, inplace = False)

In [ ]:
cnes_fato_estabelecimento

,co_unidade,co_cnes,id_razao_social,id_nome_fantasia,id_nome_logradouro,numero_endereço,id_nome_bairro,num_cep,id_atividade,data_expedicao,tipo_unidade,data_ultima_atualizacao,motivo_desativo,latitude,longitude,id_natureza_juridica,sempre_aberto,conexao_internet,tipo_unidade,gestao,id_tipo_estabelecimento,id_atividade_principal
0,SP00003509205000001329730000101,2825287,5815,5117,6547,929,0,1530000,4,NaN,4,09/09/2003,8.0,NaN,NaN,2000.0,NaN,NaN,NaN,M,NaN,NaN
1,3550309902198,9902198,10906,3324,5768,766,1,4508031,4,28-ago-2019 00:00:00,36,04/09/2019,NaN,-23.590627,-46.6682989,2240.0,N,S,NaN,M,16.0,1.0
2,3550309902309,9902309,12745,20258,4498,401,2,5406050,4,21-nov-2019 00:00:00,22,24/11/2019,NaN,-23.5592723,-46.677217,2240.0,N,S,NaN,M,16.0,1.0
3,3550300868663,868663,15346,15676,7377,126,3,5569120,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6179578,-46.5353544,4000.0,N,S,NaN,M,16.0,1.0
4,3550300868655,868655,20554,15082,5891,837,4,5717270,4,13-ago-2021 00:00:00,22,19/08/2021,NaN,-23.6349407,-46.7271941,4000.0,N,S,NaN,M,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26137,3550306749755,6749755,19076,23265,1207,191,3417,4304010,4,01-mai-2010 00:00:00,22,05/02/2019,8.0,NaN,NaN,2240.0,NaN,S,NaN,M,16.0,1.0
26138,3550306749771,6749771,21088,15185,4081,25,14,1047020,4,22-mar-2011 00:00:00,22,28/04/2011,8.0,NaN,NaN,4000.0,NaN,NaN,NaN,M,16.0,1.0
26139,3550306751040,6751040,22652,15599,3951,1770,124,4602006,4,06-jan-2009 00:00:00,22,13/02/2019,8.0,NaN,NaN,2062.0,NaN,S,NaN,M,16.0,1.0
26140,3550306751008,6751008,22480,25393,4797,229,1,4544000,4,NaN,36,20/08/2021,NaN,-23.5912333,-46.6765755,2062.0,N,S,NaN,M,16.0,1.0


In [ ]:
cnes_fato_estabelecimento.to_csv(r'cnes_fato_estabelecimento.csv', index = False)